In [7]:
import azure_utils
import os

In [8]:
pestfiles_dir = "path_to_pest_files"

azurefiles_dir = "path_to_azure_script_files"  #does not have to be created beforehand

if not os.path.exists(azurefiles_dir):
    os.makedirs(azurefiles_dir)

In [9]:
with open("Acs.dat","r") as f: # file with the connection string in it.
    acs = f.readline()
f.close()

with open("As.dat","r") as f:
    subscription_id = f.readline() # file with the subscription id in it.
f.close()

#create a config file for later use

with open(os.path.join(azurefiles_dir,"config.yaml"),"w") as g:
    g.write('azure_storage_connectionstring: "{0}"\n'.format(acs))
    g.write('storage_account_name: "pestfiles"\n')
    g.write('input_container_name: "pestinputs"\n')
    g.write('scripts_container_name: "scripts"\n')

Load the config file

In [10]:
config = azure_utils.load_config(azurefiles_dir)

Delete the existing files in both containers (pestinputs, scripts)

In [ ]:
# This step is optional if you want to delete previous files in the containers

azure_utils.delete_blobs(config["azure_storage_connectionstring"],config["scripts_container_name"])
azure_utils.delete_blobs(config["azure_storage_connectionstring"],config["input_container_name"])

Define the azure resource group and location

In [11]:
resource_group_name = 'pesthp'
location = 'eastus'
storage_account_name = 'pestfiles'

Define the Network and IP address names

In [12]:
vnet_name = "pesthp-vnet"
subnet_name = "pesthp-subnet"
ip_name = "pesthp-ip"
nsg_name = "pesthp_nsg"
ip_config_name = "pesthp-ip-config"
nic_name = "pesthp-nic"
hpmanager_script_name="manager_script.ps1"
hpagent_script_name="agent_script.ps1"

In [13]:
pst_final = "mnsda_ies_1.pst"

if __name__ == '__main__':
    
    man_script = azure_utils.gen_hpmanager_script(azurefiles_dir,
                    config["storage_account_name"],
                    config["input_container_name"],
                    config["azure_storage_connectionstring"],
                    pst_final,
                    4004,hpmanager_script_name,"pestpp-ies.exe")
    
    #Create the network_client and compute cliente objects
    network_client = azure_utils.create_network_client(subscription_id)
    compute_client = azure_utils.create_compute_client(subscription_id)
    
    vnet_result = azure_utils.create_vnet(network_client,resource_group_name,location,vnet_name)
    subnet_result = azure_utils.create_subnet(resource_group_name,network_client,vnet_result.name,subnet_name)
    public_ip_address = azure_utils.create_public_ip_address(resource_group_name,location,network_client,ip_name)
    nsg_result = azure_utils.create_nsg(resource_group_name,location,network_client,nsg_name)
    manager_nic = azure_utils.create_nic(resource_group_name,location,network_client,
                subnet_result,nsg_result,nic_name,ip_config_name,
                public_ip_address)
    
    agent_script = azure_utils.gen_hpagent_script(azurefiles_dir,
                    config["storage_account_name"],
                    config["input_container_name"],
                    config["azure_storage_connectionstring"],
                    2,pst_final,manager_nic[1],4004,hpagent_script_name,"pestpp-ies.exe")
    
    script_files=azure_utils.get_files(azurefiles_dir)
    pest_input_files=azure_utils.get_files(pestfiles_dir)
    azure_utils.upload_blobs(script_files,config["azure_storage_connectionstring"],config["scripts_container_name"])
    azure_utils.upload_blobs(pest_input_files,config["azure_storage_connectionstring"],config["input_container_name"])
    
    vm_name = "managerVM"
    vm_size = "Standard_F16s_v2"
    username = "manageruser"
    password = "ChangePa$$w0rd21"

    manager_vm = azure_utils.create_vm(resource_group_name,location,compute_client,manager_nic[0],
                                    vm_name,vm_size,username,password)
    
    # Add extension to vm
    ext_name = 'customscript_hp'
    myExecutionCommand = 'powershell.exe -ExecutionPolicy Unrestricted -File '+hpmanager_script_name
    script_location = 'https://'+storage_account_name+'.blob.core.windows.net/'+config["scripts_container_name"]+'/'+hpmanager_script_name

    hpmanager_custom_script=azure_utils.add_custom_script_extension_to_vm(resource_group_name,
                                                                      location,compute_client,
                                                                      vm_name,ext_name,
                                                                      myExecutionCommand,
                                                                      script_location)

    
# for agents

    vm_size = "Standard_F2s_v2"
    ext_name = 'customscript_hp'
    myExecutionCommand = 'powershell.exe -ExecutionPolicy Unrestricted -File '+hpagent_script_name
    script_location = 'https://'+storage_account_name+'.blob.core.windows.net/'+config["scripts_container_name"]+'/'+hpagent_script_name

    for iagent in range(98):
        nic_name = "pesthp-nic"+str(iagent+1)
        vm_name = "agentVM"+str(iagent+1)
        username = "agentuser"+str(iagent+1)
        password = "ChangePa$$w0rd21"
        manager_nic = azure_utils.create_nic(resource_group_name,location,network_client,
                subnet_result,nsg_result,nic_name,ip_config_name)
    
        agent_vm = azure_utils.create_vm(resource_group_name,location,compute_client,manager_nic[0],
                                    vm_name,vm_size,username,password)
    
        hpmanager_custom_script=azure_utils.add_custom_script_extension_to_vm(resource_group_name,
                                                                      location,compute_client,
                                                                      vm_name,ext_name,
                                                                      myExecutionCommand,
                                                                      script_location)


Provisioned virtual network pesthp-vnet with address prefixes ['10.0.0.0/16']
Provisioned virtual subnet pesthp-subnet with address prefix 10.0.0.0/24
Provisioned public IP address pesthp-ip with address 40.117.234.105
Provisioned network security group pesthp_nsg
Provisioned network interface client pesthp-nic
Uploading files to blob storage
.\mnsda_ies\pest_runs\azure_files\agent_script.ps1
agent_script.ps1 uploaded to blob storage
.\mnsda_ies\pest_runs\azure_files\config.yaml
config.yaml uploaded to blob storage
.\mnsda_ies\pest_runs\azure_files\manager_script.ps1
manager_script.ps1 uploaded to blob storage
Uploading files to blob storage

Provisioning manager virtual machine managerVM; this operation might take a few minutes.
Provisioned manager virtual machine managerVM
Provisioned network interface client pesthp-nic1

Provisioning manager virtual machine agentVM1; this operation might take a few minutes.
Provisioned manager virtual machine agentVM1
